In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from PIL import Image
import os
import numpy as np 
import torchvision.transforms.functional as F
from tqdm.auto import tqdm

# Custom dataset class to load original and rotated images
class RotationDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.all_imgs_paths = []
        dir_list = [os.path.join(self.root_dir, directory) for directory in os.listdir(self.root_dir)]
        for dir_name in dir_list:
            self.all_imgs_paths.extend([os.path.join(dir_name, img_name) for img_name in os.listdir(dir_name)])
        

    def __len__(self):
        return len(self.all_imgs_paths)

    def __getitem__(self, idx):
        img_path = self.all_imgs_paths[idx]
        image = Image.open(img_path).convert("RGB")
        original_image = self.transform(image)
        label = 0
        return original_image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define transformations
def rotate_90(image):
    return F.rotate(image, 90)

def rotate_180(image):
    return F.rotate(image, 180)

def rotate_270(image):
    return F.rotate(image, 270)





In [2]:
# Load dataset using custom dataset class
train_dataset = RotationDataset(root_dir='imagenette2/train', transform=transform)
# Apply transformations to create rotated datasets
rotated_90_dataset = [(rotate_90(image), 1) for image, label in tqdm(train_dataset)]
rotated_180_dataset = [(rotate_180(image), 2) for image, label in tqdm(train_dataset)]
rotated_270_dataset = [(rotate_270(image), 3) for image, label in tqdm(train_dataset)]

# Concatenate all datasets
concatenated_dataset = ConcatDataset([train_dataset, 
                                      rotated_90_dataset, 
                                      rotated_180_dataset, 
                                      rotated_270_dataset])

# Create a DataLoader to handle the concatenated dataset
dataloader = DataLoader(concatenated_dataset, batch_size=32, shuffle=True)


# validation dataset
val_dataset = RotationDataset(root_dir='imagenette2/val', transform=transform)
# Apply transformations to create rotated datasets
rotated_90_val_dataset = [(rotate_90(image), 1) for image, label in tqdm(val_dataset)]
rotated_180_val_dataset = [(rotate_180(image), 2) for image, label in tqdm(val_dataset)]
rotated_270_val_dataset = [(rotate_270(image), 3) for image, label in tqdm(val_dataset)]

# Concatenate all datasets
concatenated_val_dataset = ConcatDataset([val_dataset, 
                                      rotated_90_val_dataset, 
                                      rotated_180_val_dataset, 
                                      rotated_270_val_dataset])

# Create a DataLoader to handle the concatenated dataset
val_dataloader = DataLoader(concatenated_val_dataset, batch_size=32, shuffle=True)



  0%|          | 0/9469 [00:00<?, ?it/s]

  0%|          | 0/9469 [00:00<?, ?it/s]

  0%|          | 0/9469 [00:00<?, ?it/s]

  0%|          | 0/3925 [00:00<?, ?it/s]

  0%|          | 0/3925 [00:00<?, ?it/s]

  0%|          | 0/3925 [00:00<?, ?it/s]

In [3]:
import time

import lightning as L
import torch
import torch.nn.functional as F
import torchmetrics
from torchvision import transforms
from watermark import watermark
import torch.nn as nn



class PyTorchCNN(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(128 * 7 * 7, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.cnn_layers(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


def train(num_epochs, model, optimizer, scheduler, train_loader, val_loader, device):

    for epoch in range(num_epochs):
        train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=4).to(device)

        model.train()
        for batch_idx, (features, targets) in enumerate(train_loader):
            model.train()

            features = features.to(device)
            targets = targets.to(device)

            ### FORWARD AND BACK PROP
            logits = model(features)
            loss = F.cross_entropy(logits, targets)

            optimizer.zero_grad()
            loss.backward()

            ### UPDATE MODEL PARAMETERS
            optimizer.step()
            scheduler.step()

            ### LOGGING
            if not batch_idx % 300:
                print(f"Epoch: {epoch+1:04d}/{num_epochs:04d} | Batch {batch_idx:04d}/{len(train_loader):04d} | Loss: {loss:.4f}")

            model.eval()
            with torch.no_grad():
                predicted_labels = torch.argmax(logits, 1)
                train_acc.update(predicted_labels, targets)

        ### MORE LOGGING
        model.eval()
        with torch.no_grad():
            val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=4).to(device)

            for (features, targets) in val_loader:
                features = features.to(device)
                targets = targets.to(device)
                outputs = model(features)
                predicted_labels = torch.argmax(outputs, 1)
                val_acc.update(predicted_labels, targets)

            print(f"Epoch: {epoch+1:04d}/{num_epochs:04d} | Train acc.: {train_acc.compute()*100:.2f}% | Val acc.: {val_acc.compute()*100:.2f}%")
            train_acc.reset(), val_acc.reset()



print(watermark(packages="torch,lightning", python=True))
print("Torch CUDA available?", torch.cuda.is_available())
device = "cuda" if torch.cuda.is_available() else "cpu"

L.seed_everything(123)

##########################
### 1 Loading the Dataset
##########################
train_loader, val_loader = dataloader, val_dataloader


#########################################
### 2 Initializing the Model
#########################################


model = PyTorchCNN(num_classes=4)
model.to(device)
############################################################

NUM_EPOCHS = 100
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
num_steps = NUM_EPOCHS * len(train_loader)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_steps)

#########################################
### 3 Finetuning
#########################################

start = time.time()
train(
    num_epochs=NUM_EPOCHS,
    model=model,
    optimizer=optimizer,
    train_loader=train_loader,
    val_loader=val_loader,
    scheduler=scheduler,
    device=device
)

end = time.time()
elapsed = end-start
print(f"Time elapsed {elapsed/60:.2f} min")
print(f"Memory used: {torch.cuda.max_memory_reserved() / 1e9:.02f} GB")



Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.20.0

torch    : 2.3.0
lightning: 2.2.4

Torch CUDA available? True


Seed set to 123


Epoch: 0001/0100 | Batch 0000/1184 | Loss: 1.3901
Epoch: 0001/0100 | Batch 0300/1184 | Loss: 1.0135
Epoch: 0001/0100 | Batch 0600/1184 | Loss: 1.1674
Epoch: 0001/0100 | Batch 0900/1184 | Loss: 1.4280
Epoch: 0001/0100 | Train acc.: 45.08% | Val acc.: 49.36%
Epoch: 0002/0100 | Batch 0000/1184 | Loss: 1.1761
Epoch: 0002/0100 | Batch 0300/1184 | Loss: 1.2253
Epoch: 0002/0100 | Batch 0600/1184 | Loss: 1.2709
Epoch: 0002/0100 | Batch 0900/1184 | Loss: 1.0439
Epoch: 0002/0100 | Train acc.: 49.00% | Val acc.: 49.88%
Epoch: 0003/0100 | Batch 0000/1184 | Loss: 1.1471
Epoch: 0003/0100 | Batch 0300/1184 | Loss: 1.3531
Epoch: 0003/0100 | Batch 0600/1184 | Loss: 1.1270
Epoch: 0003/0100 | Batch 0900/1184 | Loss: 1.0715
Epoch: 0003/0100 | Train acc.: 50.52% | Val acc.: 50.97%
Epoch: 0004/0100 | Batch 0000/1184 | Loss: 0.9851
Epoch: 0004/0100 | Batch 0300/1184 | Loss: 0.9874
Epoch: 0004/0100 | Batch 0600/1184 | Loss: 1.1966
Epoch: 0004/0100 | Batch 0900/1184 | Loss: 1.1087
Epoch: 0004/0100 | Train acc.

KeyboardInterrupt: 

In [ ]:
print(f"Memory used: {torch.cuda.max_memory_reserved() / 1e9:.02f} GB")
